In [308]:
import pandas as pd
import numpy as np

import category_encoders as ce

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import r2_score, mean_squared_error, make_scorer
import seaborn as sns#seaborn

In [309]:
import sklearn.linear_model
import sklearn.model_selection
import sklearn.metrics
import sklearn.preprocessing
import sklearn.feature_extraction


%matplotlib inline

import matplotlib.pyplot as plt
from pprint import pformat
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

In [310]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [311]:
def train_validate_report(model, X_train, y_train, X_valid, y_valid, feature_names):
    model.fit(X_train, y_train)
    predict = model.predict(X_valid);
    print('RMSE:' , np.sqrt(mean_squared_error(y_valid, predict)))
    print('R^2:' , r2_score(y_valid, predict))

    #print(pd.DataFrame(model.coef_, feature_names, columns=['coef']).sort_values(by='coef', ascending=False))

In [312]:
def code_myohe(data, feature):
    feature = str(feature)
    for i in data[feature].unique():
        i = str(i)
        data[feature + '=' + i] = (data[feature] == i).astype(int)
    data = data.drop(feature, axis=1)
    return data

def transofrm_categorial_columns(data):
    temp_df = data.copy(deep=True)
    categorical_columns = data.columns[data.dtypes == 'object']
    for column_name in categorical_columns:
        temp_df = code_myohe(temp_df, column_name)
    return temp_df

In [313]:
linreg = LinearRegression()
lasso = Lasso(random_state = 42)
ridge = Ridge(random_state = 42)
lasso_cv = LassoCV(random_state = 42)
ridge_cv = RidgeCV()

In [314]:
def code_myohe(data, feature):
    feature = str(feature)
    for i in data[feature].unique():
        i = str(i)
        data[feature + '=' + i] = (data[feature] == i).astype(float)
    data = data.drop(feature, axis=1)
    return data

def transofrm_categorial_columns(data):
    temp_df = data.copy(deep=True)
    categorical_columns = data.columns[data.dtypes == 'object']
    for column_name in categorical_columns:
        temp_df = code_myohe(temp_df, column_name)
    return temp_df

In [315]:
def prepare_dataset(train, test):
    train = train.copy(); test = test.copy()
    
    dataset = pd.concat([train, test], keys=['train', 'test'])
    ##dataset['TotalSF'] = dataset['TotalBsmtSF'] + dataset['1stFlrSF'] + dataset['2ndFlrSF']
    ##dataset.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)
    
    #### удалим столбцы, где слишком много пропусков
    ##nans_dataset = dataset.isna().sum(axis=0).sort_values(ascending=False) / len(dataset)
    ##many_missing = list(nans_dataset[nans_dataset>0.15].index)
    ##dataset.drop(columns=many_missing, inplace=True)
    
    # заполняем пропуски
    #for col in ('GarageType', 'GarageFinish', 'GarageQual', 
    #            'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    #    dataset[col] = dataset[col].fillna('NAN')
    
    for col in dataset.columns:
        if dataset[col].dtype == 'object':
            dataset[col] = dataset[col].fillna(dataset[col].mode()[0])
        else:
            dataset[col] = dataset[col].fillna(dataset[col].mean())
            
            
    ### стандартизация        
    ##num_cols = dataset.select_dtypes(['int64', 'float64']).columns
    ##dataset[num_cols] = StandardScaler().fit_transform(dataset[num_cols])

            
    train_features = dataset.loc['train']
    test_features = dataset.loc['test']
    
    # преобразуем категориальные переменные при помощи TargetEncoder
    ce_target = ce.TargetEncoder()
    train_features = ce_target.fit_transform(train_features, train['SalePrice'])
    test_features = ce_target.transform(test_features)
    ##train_features = transofrm_categorial_columns(train_features)
    ##test_features = transofrm_categorial_columns(test_features)
    
    return train_features, test_features

In [316]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [317]:
trainP, testP = prepare_dataset(train, test)
trainP = trainP.drop(['Id'], axis=1);
testP = testP.drop(['Id'], axis=1);

In [318]:
X = trainP
y = np.log(train['SalePrice'])

In [319]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1547)

In [320]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.linear_model import LassoCV, RidgeCV

In [321]:
linreg = LinearRegression()
lasso = Lasso(random_state = 1547)
ridge = Ridge(random_state = 1547)
lasso_cv = LassoCV(random_state = 1547)
ridge_cv = RidgeCV()

In [322]:
def train_validate_report(model, X_train, y_train, X_valid, y_valid, feature_names):
    model.fit(X_train, y_train)
    predict = model.predict(X_valid);
    print('RMSE:' , np.sqrt(mean_squared_error(y_valid, predict)))
    print('R^2:' , r2_score(y_valid, predict))
    #print(pd.DataFrame(model.coef_, feature_names, columns=['coef']).sort_values(by='coef', ascending=False))

In [323]:
train_validate_report(linreg, X_train, y_train, X_test, y_test, X_train.columns)

RMSE: 0.11665357812483661
R^2: 0.9122738288172421


In [324]:
train_validate_report(lasso, X_train, y_train, X_test, y_test, X_train.columns)

RMSE: 0.11019840046078515
R^2: 0.9217140858083812


In [325]:
train_validate_report(ridge, X_train, y_train, X_test, y_test, X_train.columns)

RMSE: 0.11658990610479647
R^2: 0.9123695683259885


In [326]:
train_validate_report(lasso_cv, X_train, y_train, X_test, y_test, X_train.columns)

RMSE: 0.11594136671551697
R^2: 0.9133417574831512


In [327]:
train_validate_report(ridge_cv, X_train, y_train, X_test, y_test, X_train.columns)

RMSE: 0.11674168881262012
R^2: 0.9121412562452472


In [328]:
predictions = lasso.predict(testP)
test_id = test['Id']
res = pd.DataFrame({"Id": test_id, "SalePrice": predictions})
res.to_csv('predict_linear.csv', index=False)

In [329]:
res.head()

,Id,SalePrice
0,1461,11.892095
1,1462,12.012517
2,1463,12.114008
3,1464,12.158800
4,1465,12.090195
